In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform,data
%matplotlib inline
import sys,os,caffe
import cv2
import math
import time
from scipy.ndimage import gaussian_filter
from numpy.lib.stride_tricks import as_strided as ast

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# RGB -> YCbCr
def rgb2ycbcr(rgb):
    m = np.array([[ 65.481, 128.553, 24.966],
                  [-37.797, -74.203, 112],
                  [ 112, -93.786, -18.214]])
    shape = rgb.shape
    if len(shape) == 3:
        rgb = rgb.reshape((shape[0] * shape[1], 3))
    ycbcr = np.dot(rgb, m.transpose() / 255.)
    ycbcr[:,0] += 16.
    ycbcr[:,1:] += 128.
    return ycbcr.reshape(shape)

def psnr(img1, img2, boundary):
    HR = img1[boundary:img1.shape[0]-boundary,boundary:img1.shape[1]-boundary]
    LR = img2[boundary:img2.shape[0]-boundary,boundary:img2.shape[1]-boundary]
    mse = np.mean( (HR - LR) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def accuracy(img1, img2, boundary):
    HR = img1[boundary:img1.shape[0]-boundary,boundary:img1.shape[1]-boundary]
    LR = img2[boundary:img2.shape[0]-boundary,boundary:img2.shape[1]-boundary]
    error = HR - LR
    num_wrong = np.count_nonzero(error)
    num_all = img1.shape[0] * img1.shape[1]
    
    return (num_all - num_wrong) / num_all


caffe_root = 'XXX/caffe/' 
sys.path.insert(0, caffe_root + 'python')
os.chdir(caffe_root)

    
caffe.set_mode_gpu()


#######---------------- Coarse SR----------------------###################
##########################################################################
model_def = caffe_root + 'XXX/STSR_face_Deploy.prototxt'
image_root = caffe_root + 'XXX/Helen/'

Model_root = 'XXX/Face_Coarse_SR.caffemodel'

model_weights = caffe_root + Model_root

net = caffe.Net(model_def, model_weights, caffe.TEST)

dis = 32
psnr_ave = 0
start = time.time()
swap = (2, 0, 1)
channel_swap = (0, 2, 3, 1)
for i in range(0,50,1):
    LR = cv2.imread(image_root + "LR/LR_" + str(i+1) + ".bmp")
    HR = cv2.imread(image_root + "HR/HR_" + str(i+1) + ".bmp")
    HR_float = np.float32(HR)
    LR_float = np.float32(LR)
    shape = HR.shape
    shape_s = LR.shape
    SR_final = np.zeros((shape[0],shape[1],3)).astype(np.float32)
    LR_rot = np.zeros((shape_s[0],shape_s[1],3)).astype(np.float32)
    
    for d in range(0,4,1):
        LR_rot = np.rot90(LR_float,d)
        LR = LR_rot.transpose(swap)
        net.blobs['data'].data[...] = LR/255
        net.forward()
        out_image = net.blobs['SR_final_s3'].data
        out_image = out_image.transpose(channel_swap)
        SR = out_image[0,:,:,:]
        SR_final = SR_final + np.rot90(SR,4-d)*0.125*255
        
    for d in range(0,4,1):
        LR_rot = np.rot90(np.fliplr(LR_float),d)
        LR = LR_rot.transpose(swap)
        net.blobs['data'].data[...] = LR/255
        net.forward()
        out_image = net.blobs['SR_final_s3'].data
        out_image = out_image.transpose(channel_swap)
        SR = out_image[0,:,:,:]
        SR_final = SR_final + np.fliplr(np.rot90(SR,4-d))*0.125*255
    cv2.imwrite(image_root + "SR/SR_" + str(i+1) + ".bmp",SR_final)
    
    HR_YUV = rgb2ycbcr(HR_float)
    SR_YUV = rgb2ycbcr(SR_final)
    HR_Y = HR_YUV[:,:,0]
    SR_Y = SR_YUV[:,:,0]
    value = psnr(SR_Y, HR_Y, 8)
    psnr_ave = psnr_ave + value/50
    
end = time.time()
cost = end - start
print "time cost is ", cost            
print "average PSNR is ", psnr_ave


#######---------------- Face Segmentation----------------------###################
##################################################################################

model_def = caffe_root + 'XXX/HCRF_s2_Deploy.prototxt'
image_root = caffe_root + 'XXX/Segmentation/'

Model_root = 'XXX/Face_Segmentation.caffemodel'

model_weights = caffe_root + Model_root

net = caffe.Net(model_def, model_weights, caffe.TEST)
    
size = 128

Segmentation_Map = np.zeros((size,size)).astype(np.double)
color_mask = np.zeros((size,size,3)).astype(np.double)
Seg_map = np.zeros((size,size)).astype(np.double)
accuracy_ave = 0
start = time.time()
for i in range(0,50,1):
    HR = cv2.imread(image_root + "SR/SR_" + str(i+1) + ".bmp")
    HR_float = np.float32(HR)
    #GT = cv2.imread(image_root + "Mask_all/mask_" + str(i+1) + ".bmp",0)
    #GT_float = np.float32(GT)
    
    
    swap = (2, 0, 1)
    channel_swap = (1, 2, 0)
    Seg_final = np.zeros((size,size,6)).astype(np.double)
    for d in range(0,1,1):
        HR_float = np.rot90(np.float32(HR),d)
        HR_float = HR_float.transpose(swap)
        net.blobs['data'].data[...] = HR_float/255
        net.forward()
        out_image = net.blobs['prob'].data[0,:,:,:]

        Seg_final = Seg_final + np.rot90(out_image.transpose(channel_swap),4-d)*1
        
    Seg_map = np.argmax(Seg_final, axis=2)
    
    for m in range(0,size,1):
        for n in range(0,size,1):
            value = Seg_map[m,n]
            Segmentation_Map[m,n] = value
            
            if value == 0:
                color_mask[m,n,0] = 0
                color_mask[m,n,1] = 0
                color_mask[m,n,2] = 0
            elif value == 1:
                color_mask[m,n,2] = 255
                color_mask[m,n,1] = 222
                color_mask[m,n,0] = 173
            elif value == 2:
                color_mask[m,n,2] = 176
                color_mask[m,n,1] = 196
                color_mask[m,n,0] = 222
            elif value == 3:
                color_mask[m,n,2] = 85
                color_mask[m,n,1] = 26
                color_mask[m,n,0] = 139
            elif value == 4:
                color_mask[m,n,2] = 139
                color_mask[m,n,1] = 129
                color_mask[m,n,0] = 76
            elif value == 5:
                color_mask[m,n,2] = 255
                color_mask[m,n,1] = 0
                color_mask[m,n,0] = 0
    
    cv2.imwrite(image_root + "output/parse_" + str(i+1) + ".bmp",color_mask)
    cv2.imwrite(image_root + "output_label/mask_" + str(i+1) + ".bmp",Segmentation_Map)
    
    Mask = np.ones((size,size)).astype(np.double)
    Mask = np.ones((size,size)).astype(np.double) - Mask*(GT_float<2)
    '''
    error = GT_float * Mask - Segmentation_Map * Mask
    num_wrong = np.count_nonzero(error)
    num_all = np.count_nonzero(GT_float * Mask)
    value = np.double(num_all - num_wrong)/np.double(num_all)
    accuracy_ave = accuracy_ave + value/50.0
    '''
    
